In [1]:
# Mounting google drive
from google.colab import drive

drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [27]:
# Imports

import numpy as np
import librosa
from sklearn.model_selection import train_test_split
import os
import math
import json
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

## **Extracting and storing MFCCs from each audio signal**

In [7]:
# Defining constants

dataset = '/content/gdrive/MyDrive/Data/genres_original'
sample_rate = 22050
duration = 30 #secs
samples_per_track = sample_rate * duration




In [ ]:
# Funtion to extract mfccs from each segment of each audio file and store them in a json file

def mfcc_extract(dataset, num_mfcc = 13, n_fft = 2048, hop_length = 512, num_segments = 5):

  data = { "classes":[], "mfccs":[],"labels":[] }

  samples_per_segment = int(samples_per_track/num_segments)
  num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

  for i,(root,dirs,files) in enumerate(os.walk(dataset)):

    if root is not dataset:
      
      # Extracting class name from directory path and storing it in the dictionary
      label = root.split('/')[-1]
      data['classes'].append(label)

      # Extracting MFCCs from each audio file segment
      for f in files:

        file_path = os.path.join(root,f)
        signal,sr = librosa.load(file_path,sr = sample_rate)

        for s in range(num_segments):

          # Find starting and ending points of each segment in the audio file
          start = samples_per_segment * s
          end = start + samples_per_segment

          # Extract mfcc for current segment 
          mfcc = librosa.feature.mfcc(signal[start:end], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length).T

          # Ensuring that shape of mfccs is fixed to avoid problems when feeding the data into the network
          if len(mfcc) == num_mfcc_vectors_per_segment:
              data["mfccs"].append(mfcc.tolist())
              data["labels"].append(i-1)
  
  with open('/content/gdrive/MyDrive/Data/mfcc.json','w') as f:
    json.dump(data, f)



mfcc_extract(dataset)

### **Artificial Neural Network for Music genre classification**

In [4]:
# Loading the data from the mfcc.json file

with open('/content/gdrive/MyDrive/Data/mfcc.json','r') as f:
  data = json.load(f)

# Removing unwanted classes 
data['classes'].remove('.ipynb_checkpoints')
print(data['classes'])
np.unique(data['labels'])


['rock', 'blues', 'reggae', 'disco', 'pop', 'hiphop', 'metal', 'country', 'classical', 'jazz']


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
# Splitting data into train and test sets

X = np.array(data['mfccs'])
y = np.array(data['labels'])

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

#Printing shapes for sanity check
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)





(3495, 259, 13)
(1498, 259, 13)
(3495,)
(1498,)


In [ ]:
# Building the model

model = keras.Sequential([
        
        keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 3367)              0         
_________________________________________________________________
dense_39 (Dense)             (None, 512)               1724416   
_________________________________________________________________
dense_40 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_41 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_42 (Dense)             (None, 10)                650       
Total params: 1,872,842
Trainable params: 1,872,842
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Training the model

model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=100)

Epoch 1/100
110/110 [==============================] - 3s 28ms/step - loss: 0.9901 - accuracy: 0.8438 - val_loss: 2.8188 - val_accuracy: 0.5327
Epoch 2/100
110/110 [==============================] - 3s 27ms/step - loss: 0.8350 - accuracy: 0.8761 - val_loss: 2.8233 - val_accuracy: 0.5467
Epoch 3/100
110/110 [==============================] - 3s 27ms/step - loss: 1.0674 - accuracy: 0.8100 - val_loss: 2.8490 - val_accuracy: 0.5073
Epoch 4/100
110/110 [==============================] - 3s 27ms/step - loss: 0.9122 - accuracy: 0.8658 - val_loss: 2.7679 - val_accuracy: 0.5427
Epoch 5/100
110/110 [==============================] - 3s 27ms/step - loss: 0.7336 - accuracy: 0.9182 - val_loss: 2.8339 - val_accuracy: 0.5607
Epoch 6/100
110/110 [==============================] - 3s 27ms/step - loss: 0.9359 - accuracy: 0.8506 - val_loss: 2.6290 - val_accuracy: 0.5274
Epoch 7/100
110/110 [==============================] - 3s 27ms/step - loss: 0.7426 - accuracy: 0.9087 - val_loss: 2.7750 - val_accuracy:

## **CNN for Music genre classification**

In [10]:
# Obtaining the inputs and outputs

X = np.array(data['mfccs'])
y = np.array(data['labels'])

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

# Adding another dimension to the inputs as CNNs accept inputs of shape [w,h,num_channels]
X_train = X_train[... , np.newaxis]
X_test = X_test[... , np.newaxis]

# Sanity check
print(X_test.shape)
print(X_train.shape)
print(y_test.shape)
print(y_train.shape)


(1498, 259, 13, 1)
(3495, 259, 13, 1)
(1498,)
(3495,)


In [38]:
# Building the CNN model

cnn_model = keras.Sequential([
                              
            keras.layers.Conv2D(32,[3,3],activation='relu',input_shape = [X_train.shape[1],X_train.shape[2],X_train.shape[3]]),
            keras.layers.MaxPooling2D([3,3],strides=(2,2),padding='same'),
            keras.layers.BatchNormalization(),

            keras.layers.Conv2D(64,[2,2],activation='relu'),
            keras.layers.MaxPooling2D([3,3],strides=[2,2],padding='same'),
            keras.layers.BatchNormalization(),

            keras.layers.Conv2D(128,[2,2],activation='relu'),
            keras.layers.MaxPooling2D([2,2],strides=[2,2],padding='same'),
            keras.layers.BatchNormalization(),

            keras.layers.Flatten(),
            

            keras.layers.Dense(64,activation='relu',kernel_regularizer= regularizers.l2(1e-4)),
            keras.layers.Dropout(0.4),

            keras.layers.Dense(10,activation='softmax')

])
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
cnn_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

cnn_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 257, 11, 32)       320       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 129, 6, 32)        0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 129, 6, 32)        128       
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 128, 5, 64)        8256      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 64, 3, 64)         0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 64, 3, 64)         256       
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 63, 2, 128)       

In [39]:
# Training the model 

cnn_model.fit(X_train,y_train,validation_split=0.2,batch_size=32,epochs=15) 

Epoch 1/15
88/88 [==============================] - 14s 153ms/step - loss: 2.4934 - accuracy: 0.3516 - val_loss: 2.4485 - val_accuracy: 0.2890
Epoch 2/15
88/88 [==============================] - 13s 151ms/step - loss: 1.4896 - accuracy: 0.5422 - val_loss: 1.5984 - val_accuracy: 0.4435
Epoch 3/15
88/88 [==============================] - 13s 152ms/step - loss: 1.0757 - accuracy: 0.6352 - val_loss: 1.2408 - val_accuracy: 0.5837
Epoch 4/15
88/88 [==============================] - 13s 152ms/step - loss: 0.8271 - accuracy: 0.7092 - val_loss: 1.0996 - val_accuracy: 0.6209
Epoch 5/15
88/88 [==============================] - 14s 154ms/step - loss: 0.6548 - accuracy: 0.7761 - val_loss: 1.2659 - val_accuracy: 0.6309
Epoch 6/15
88/88 [==============================] - 13s 152ms/step - loss: 0.5454 - accuracy: 0.8119 - val_loss: 1.3095 - val_accuracy: 0.6037
Epoch 7/15
88/88 [==============================] - 13s 152ms/step - loss: 0.4410 - accuracy: 0.8598 - val_loss: 1.1007 - val_accuracy: 0.6781

In [40]:
# Evaluating the CNN model on the test set

test_accuracy = cnn_model.evaluate(X_test,y_test,verbose=2)

47/47 - 1s - loss: 1.4766 - accuracy: 0.6409
